<a href="https://colab.research.google.com/github/elkinkazan/itis-2025-ai-tools/blob/main/tasks/Task1_PyTorch_Iliasova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# DO NOT MODIFY  !!!
# DO NOT EXECUTE !!!
!pip install --upgrade gspread pandas google-auth
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
from IPython.display import display
import random
# Authenticate and create the PyDrive client.
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)



In [24]:
# FILL THIS
student_name = "ELVIRA ILIASOVA"
group_id = "11-451"


In [ ]:
# DO NOT MODIFY  !!!
# DO NOT EXECUTE !!!
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1Kfxj2eDFl7xQnXw7Fpb9bwghc65o8xf--VpNxrdWHaY/edit?gid=0#gid=0"
sh = gc.open_by_url(SPREADSHEET_URL)
worksheet = sh.sheet1

# Ensure header row exists
if not worksheet.get_all_values():
    worksheet.append_row(["Student Name", "Group","TaskID", "Score"])


In [25]:
# MAIN NOTEBOOK GOES HERE
task_id = "Task1_PyTorch"
score = 0
max_score = 15

In [26]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [27]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


In [29]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# Modelling

In [31]:
# mps for Mac
global device
device = "cuda"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To train a model, we need a loss function and an optimizer.

In [35]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.

In [36]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [37]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return  100*correct, test_loss

In [38]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Epoch 1
-------------------------------
loss: 2.299617  [   64/60000]
loss: 2.301858  [ 6464/60000]
loss: 2.283638  [12864/60000]
loss: 2.273639  [19264/60000]
loss: 2.259116  [25664/60000]
loss: 2.229350  [32064/60000]
loss: 2.235443  [38464/60000]
loss: 2.204156  [44864/60000]
loss: 2.188205  [51264/60000]
loss: 2.168362  [57664/60000]
Test Error: 
 Accuracy: 37.5%, Avg loss: 2.163874 

Epoch 2
-------------------------------
loss: 2.167197  [   64/60000]
loss: 2.174447  [ 6464/60000]
loss: 2.116549  [12864/60000]
loss: 2.129075  [19264/60000]
loss: 2.091422  [25664/60000]
loss: 2.029573  [32064/60000]
loss: 2.057814  [38464/60000]
loss: 1.985988  [44864/60000]
loss: 1.974467  [51264/60000]
loss: 1.923897  [57664/60000]
Test Error: 
 Accuracy: 56.3%, Avg loss: 1.917178 

Epoch 3
-------------------------------
loss: 1.942154  [   64/60000]
loss: 1.936143  [ 6464/60000]
loss: 1.809971  [12864/60000]
loss: 1.848532  [19264/60000]
loss: 1.757295  [25664/60000]
loss: 1.692371  [32064/600

# Task 1
Build graphic for the learning process.

1. Load/Save models

In [39]:
def createModel():
  # YOUR CODE START 2 lines
  # create model
  model = NeuralNetwork().to(device)

  return model
  # YOUR CODE END

In [40]:
def checkClasses(model):
  classes = [
      "T-shirt/top",
      "Trouser",
      "Pullover",
      "Dress",
      "Coat",
      "Sandal",
      "Shirt",
      "Sneaker",
      "Bag",
      "Ankle boot",
  ]

  model.eval()
  # YOUR CODE START 4 lines
  # load 1st class
  x, y = test_data[0]
  x = x.unsqueeze(0).to(device)
  with torch.no_grad():
      # predict
      pred = model(x)
      # return predicted, actual
      predicted, actual = classes[pred.argmax(1).item()], classes[y]
  return predicted, actual

In [41]:
import unittest
import cv2
import numpy as np

class TestImageFilters(unittest.TestCase):

    def test_model_creation(self):
        # Загрузка изображения
        model = createModel()
        predicted, actual = checkClasses(model)
        self.assertEqual(predicted, actual, "Prediction wrong")
        global score
        score=score+5

# Run unit tests
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
    print(f"Student Score: {score}/{max_score}")

.
----------------------------------------------------------------------
Ran 1 test in 0.008s

OK


Student Score: 5/15


In [48]:
model = createModel()
predicted, actual = checkClasses(model)
print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


# Task 2. Improve training

In [54]:
import torch.optim as optim

# Improve Model Function
def improveModel():
    epochs = 10
    global device

    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)

        acc, loss = test(test_dataloader, model, loss_fn)

        print(f"Test Accuracy after Epoch {t+1}: {acc:.2f}%")
        print(f"Test Loss after Epoch {t+1}: {loss:.4f}")

    return acc, loss

In [55]:
import unittest
import cv2
import numpy as np

class TestImageFilters(unittest.TestCase):

    def test_model_improve(self):
        acc, loss = improveModel()
        self.assertGreaterEqual(acc, 80, "Model is not accurate")
        global score
        score=score+10

# Run unit tests
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
    print(f"Student Score: {score}/{max_score}")

Epoch 1
-------------------------------
loss: 0.303817  [   64/60000]
loss: 0.071195  [ 6464/60000]
loss: 0.071306  [12864/60000]
loss: 0.134597  [19264/60000]
loss: 0.108397  [25664/60000]
loss: 0.142220  [32064/60000]
loss: 0.080574  [38464/60000]
loss: 0.106518  [44864/60000]
loss: 0.052663  [51264/60000]
loss: 0.105620  [57664/60000]
Test Error: 
 Accuracy: 89.0%, Avg loss: 0.641071 

Test Accuracy after Epoch 1: 88.98%
Test Loss after Epoch 1: 0.6411
Epoch 2
-------------------------------
loss: 0.075702  [   64/60000]
loss: 0.083189  [ 6464/60000]
loss: 0.121197  [12864/60000]
loss: 0.098074  [19264/60000]
loss: 0.087218  [25664/60000]
loss: 0.171326  [32064/60000]
loss: 0.039350  [38464/60000]
loss: 0.129415  [44864/60000]
loss: 0.041262  [51264/60000]
loss: 0.134710  [57664/60000]
Test Error: 
 Accuracy: 88.1%, Avg loss: 0.593393 

Test Accuracy after Epoch 2: 88.12%
Test Loss after Epoch 2: 0.5934
Epoch 3
-------------------------------
loss: 0.137583  [   64/60000]
loss: 0.04

.
----------------------------------------------------------------------
Ran 1 test in 81.524s

OK


Test Error: 
 Accuracy: 88.3%, Avg loss: 0.683499 

Test Accuracy after Epoch 10: 88.30%
Test Loss after Epoch 10: 0.6835
Student Score: 35/15


In [ ]:
# DO NOT MODIFY  !!!
# DO NOT EXECUTE !!!
# Save the result to Google Sheets
worksheet.append_row([student_name,group_id, task_id, score])

print(f"Test completed! {student_name}, your score is {score}/{max_score}.")

Test completed! ELON MUSK, your score is 5/15.
